In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.offsetbox import AnchoredText
import json
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math 

In [23]:
import time

In [2]:
class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x

In [4]:
pi = math.pi
L     = 14 

In [5]:
w = 0; rad = math.radians(w); j1 = math.sin(rad); j2 = math.cos(rad)
J = [j1,j2]

In [42]:
J

[0.0, 1.0]

In [46]:
w = 60; rad = math.radians(w); j1 = math.sin(rad); j2 = math.cos(rad)
J = [j1,j2]

In [48]:
w,J

(60, [0.8660254037844386, 0.5000000000000001])

In [49]:
# Define custom graph
edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])

# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors)

In [50]:
#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))

#Exchange interactions
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

bond_operator = [
    (J[0] * mszsz).tolist(),
    (J[1] * mszsz).tolist(),
    (-J[0] * exchange).tolist(),  
    (J[1] * exchange).tolist(),
]

bond_color = [1, 2, 1, 2]

In [51]:
# Spin based Hilbert Space
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)

In [52]:
# Custom Hamiltonian operator
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

In [53]:
model = FFNN() #Neural Network

In [54]:
# We shall use an exchange Sampler which preserves the global magnetization (as this is a conserved quantity in the model)
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)

# Construct the variational state
vs = nk.vqs.MCState(sa, model, n_samples=1008) #use model

# We choose a basic, albeit important, Optimizer: the Stochastic Gradient Descent
opt = nk.optimizer.Sgd(learning_rate=0.01)

# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.01)

# We can then specify a Variational Monte Carlo object, using the Hamiltonian, sampler and optimizers chosen.
# Note that we also specify the method to learn the parameters of the wave-function: here we choose the efficient
# Stochastic reconfiguration (Sr), here in an iterative setup
gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr) #use vs

In [55]:
# We need to specify the local operators as a matrix acting on a local Hilbert space 
sf = []
sites = []
structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(0, L):
    for j in range(0, L):
        structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L

In [56]:
PARAM = w

In [57]:
# Run the optimization protocol
param_file ="log/" + str(PARAM)
gs.run(out=param_file, n_iter=600, obs={'Structure Factor': structure_factor})

100%|█| 600/600 [02:01<00:00,  4.95it/s, Energy=-18.1718-0.0022j ± 0.0073 [σ²=0.


(JsonLog('log/60', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.25739288330078125
   	Params: 0.0024275779724121094,)

In [58]:
data=json.load(open(param_file + ".log"))
iters = data['Energy']['iters']
energy=data['Energy']['Mean']['real']
sf=data['Structure Factor']['Mean']['real']

In [59]:
len(energy)

600

In [60]:
energy[-1]

-18.171792678362237

In [61]:
# RBM ansatz with alpha=1
ma = nk.models.RBM(alpha=1)

In [62]:
# Build the sampler
sa = nk.sampler.MetropolisExchange(hilbert=hi,graph=g)

# Custom Hamiltonian operator
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

# Optimizer
opt = nk.optimizer.Sgd(learning_rate=0.01)

# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.1)

# The variational state
vs = nk.vqs.MCState(sa, ma, n_samples=2000)

# The ground-state optimization loop
gs = nk.VMC(
    hamiltonian=op,
    optimizer=opt,
    preconditioner=sr,
    variational_state=vs)

print('### RBM calculation')
print('Has',vs.n_parameters,'parameters')
print('The RBM calculation took',end-start,'seconds')

### RBM calculation
Has 224 parameters
The RBM calculation took 54.30625605583191 seconds


In [63]:
# Run the optimization protocol
param_file ="log_rbm/" + str(PARAM)
start = time.time()
gs.run(out=param_file, n_iter=1200, obs={'Structure Factor': structure_factor})
end = time.time()

100%|█| 1200/1200 [01:05<00:00, 18.41it/s, Energy=-18.20200 ± 0.00033 [σ²=0.0002


In [64]:
data=json.load(open(param_file + ".log"))
# Extract the relevant information
iters_RBM = data["Energy"]["iters"]
energy_RBM = data["Energy"]["Mean"]

In [65]:
len(energy_RBM)

1200

In [66]:
energy_RBM[-1]

-18.201999740442137

In [67]:
E_gs, ket_gs = nk.exact.lanczos_ed(op, compute_eigenvectors=True)
structure_factor_gs = (ket_gs.T.conj()@structure_factor.to_linear_operator()@ket_gs).real[0,0]

In [68]:
E_gs[0]

-18.374663708562185

In [71]:
(J[0],J[1],E_gs[0], energy[-1], energy_RBM[-1])

(0.8660254037844386,
 0.5000000000000001,
 -18.374663708562185,
 -18.171792678362237,
 -18.201999740442137)